In [ ]:
import pandas
import numpy
from matplotlib import pyplot
filename = "dataframe.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = data.index.astype(float)
xmax, xmin = 2000, 1000
data = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])

In [ ]:
# 1点でベースライン補正 / baseline correction by one point
flag = False
#flag = True
x = 2000  # 補正点
y = data.iloc[:, numpy.abs(numpy.asarray(data.columns) - x).argmin()]
buff = (data.T - y).T
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# 1領域でベースライン補正 / baseline correction by one region
flag = False
#flag = True
xmax, xmin = 2000, 1900  # 補正領域
y = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)].T.min()
buff = (data.T - y).T
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# 2点でベースライン補正 / baseline correction by two points
flag = False
#flag = True
x1, x2 = 2000, 1900  # 補正点
y1 = data.iloc[:, numpy.abs(numpy.asarray(data.columns) - x1).argmin()]
y2 = data.iloc[:, numpy.abs(numpy.asarray(data.columns) - x2).argmin()]
a = (y2 - y1) / (x2 - x1)
b = y1 - a * x1
base = numpy.outer(a, data.columns) + numpy.array([b]).T
buff = pandas.DataFrame(data - base, index=data.index, columns=data.columns)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# 2領域でベースライン補正 / baseline correction by two regions
flag = False
#flag = True
xmax, xmin = 1700, 1500  # 領域1
buff = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
x1, y1 = buff.idxmin(axis=1), buff.min(axis=1)
xmax, xmin = 1300, 1100  # 領域2
buff = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
x2, y2 = buff.idxmin(axis=1), buff.min(axis=1)
a = (y2 - y1) / (x2 - x1)
b = y1 - a * x1
base = numpy.outer(a, data.columns) + numpy.array([b]).T
buff = pandas.DataFrame(data - base, index=data.index, columns=data.columns)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# rubberband baseline correction
flag = False
#flag = True
from scipy.spatial import ConvexHull
buff = data
x = data.columns
for i in range(len(data)):
	y = data.iloc[i].values
	v = ConvexHull(list(zip(x, y))).vertices
	v = numpy.roll(v, -v.argmin())
	v = v[:v.argmax()]
	buff.iloc[i] = data.iloc[i] - numpy.interp(x, x[v], y[v])
buff.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
pyplot.show()
if flag: data = buff

In [ ]:
# SNV
flag = False
#flag = True
buff = ((data.T - data.T.mean()) / data.T.std()).T
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# MSC
flag = False
#flag = True
x = numpy.matrix(data)
xc = x - x.mean(axis=1)
r = x.mean(axis=0)
rc = r - r.mean()
b = xc * rc.T * (rc * rc.T).I
buff = xc / b + r.mean()
buff = pandas.DataFrame(buff, index=data.index, columns=data.columns)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff